In [1]:
%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)

TensorFlow 1.x selected.
1.15.2


In [2]:
import os
from google.colab import drive 
drive.mount('/content/gdrive') 
os.getcwd() 

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


'/content'

In [3]:
%cd 'gdrive/My Drive/FinalProject'
os.getcwd() 

/content/gdrive/My Drive/FinalProject


'/content/gdrive/My Drive/FinalProject'

In [4]:
# !git clone https://github.com/qqwweee/keras-yolo3 # 如果之前已經下載過就可以註解掉
%cd keras-yolo3

/content/gdrive/My Drive/FinalProject/keras-yolo3


In [5]:
import xml.etree.ElementTree as ET
# 載入能夠 Parser xml 文件的 library
if not (os.path.exists("finalproject_train.txt") and os.path.exists("finalproject_valid.txt")):
  sets=[('Raccoon','200', '180'), ('Kangaroo','200','180')]
  classes = ["raccoon", "kangaroo"]
  
  def convert_annotation(class_id, image_id, list_file):
    in_file = open('../DataSet/%s/annotations/%s.xml'%(class_id, image_id))
    print('../DataSet/%s/annotations/%s.xml'%(class_id, image_id))
    tree=ET.parse(in_file)
    root = tree.getroot()

    for obj in root.iter('object'):
        difficult = obj.find('difficult').text
        cls = obj.find('name').text
        if cls not in classes or int(difficult)==1:
          continue
        cls_id = classes.index(cls)
        xmlbox = obj.find('bndbox')
        b = (int(xmlbox.find('xmin').text), int(xmlbox.find('ymin').text), int(xmlbox.find('xmax').text), int(xmlbox.find('ymax').text))
        list_file.write(" " + ",".join([str(a) for a in b]) + ',' + str(cls_id))

  annotation_path_train = 'finalproject_train.txt'
  list_file_train = open(annotation_path_train, 'w')
  print("save annotation at %s" % annotation_path_train)

  annotation_path_valid = 'finalproject_valid.txt'
  list_file_valid = open(annotation_path_valid, 'w')
  print("save annotation at %s" % annotation_path_valid)

  for class_id, class_total, train_num in sets:
    image_ids = open('../DataSet/%s/%s.txt'%(class_id, class_id)).read().strip().split()
    for image_id in image_ids: 
      print(f"image_id={image_id}\n")
      num=''.join([x for x in image_id if x.isdigit()])
      if(int(num) < int(class_total)):
        list_file_train.write('../DataSet/%s/images/%s.jpg'%(class_id, image_id))
        convert_annotation(class_id, image_id, list_file_train)
        list_file_train.write('\n')      
      else:
        list_file_valid.write('../DataSet/%s/images/%s.jpg'%(class_id, image_id))
        convert_annotation(class_id, image_id, list_file_valid)
        list_file_valid.write('\n')
  
  list_file_train.close()
  list_file_valid.close()
  print("done!")

In [6]:
import os
if not os.path.exists("model_data/yolo.h5"):
  # 下載 yolov3 的網路權重，並且把權重轉換為 keras 能夠讀取的格式
  print("Model doesn't exist, downloading...")
  os.system("wget https://pjreddie.com/media/files/yolov3.weights")
  print("Converting yolov3.weights to yolo.h5...")
  os.system("python convert.py yolov3.cfg yolov3.weights model_data/yolo.h5")
else:
  print("Model exist")

# 直接下載 VOC2007 的資料集作為範例
if not os.path.exists("VOCdevkit"):
  print("VOCdevkit doesn't exist, downloading...")
  os.system("wget http://pjreddie.com/media/files/VOCtrainval_06-Nov-2007.tar") # 下載 VOC 資料集
  os.system("tar xvf VOCtrainval_06-Nov-2007.tar") # 解壓縮資料集，會花幾分鐘
else:
  print("data exists")

Model exist
data exists


In [7]:
if not os.path.exists("model_data/yolo_weights.h5"):
  print("Converting pretrained YOLOv3 weights for training")
  os.system("python convert.py -w yolov3.cfg yolov3.weights model_data/yolo_weights.h5") 
else:
  print("Pretrained weights exists")

Pretrained weights exists


In [8]:
# 將 train.py 所需要的套件載入
import numpy as np
import keras.backend as K
from keras.layers import Input, Lambda
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

Using TensorFlow backend.


In [9]:
from yolo3.model import preprocess_true_boxes, yolo_body, tiny_yolo_body, yolo_loss
from yolo3.utils import get_random_data
from train import get_classes, get_anchors, create_model, create_tiny_model, data_generator, data_generator_wrapper

In [ ]:
pip install keras==2.1.5

In [ ]:
from tensorflow.python.ops import control_flow_ops
from PIL import ImageFile

ImageFile.LOAD_TRUNCATED_IMAGES = True

annotation_path = 'finalproject_train.txt' 
log_dir = 'logs/000/' 
classes_path = './model_data/finalproject_classes.txt'
anchors_path = './model_data/yolo_anchors.txt'
class_names = get_classes(classes_path)
num_classes = len(class_names)
anchors = get_anchors(anchors_path)

input_shape = (416,416) # multiple of 32, hw

is_tiny_version = len(anchors)==6 # default setting
if is_tiny_version:
    model = create_tiny_model(input_shape, anchors, num_classes,
        freeze_body=2, weights_path='model_data/tiny_yolo_weights.h5')
else:
    model = create_model(input_shape, anchors, num_classes, freeze_body=2, weights_path='model_data/yolo_weights.h5') 

logging = TensorBoard(log_dir=log_dir)

checkpoint = ModelCheckpoint(log_dir + 'ep{epoch:03d}-loss{loss:.3f}-val_loss{val_loss:.3f}.h5',
    monitor='val_loss', save_weights_only=True, save_best_only=True, period=3)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1)

# 分為 training 以及 validation
val_split = 0.1
with open(annotation_path) as f:
    lines = f.readlines()
np.random.seed(10101)
np.random.shuffle(lines)
np.random.seed(None)
num_val = int(len(lines)*val_split)
num_train = len(lines) - num_val

# Train with frozen layers first, to get a stable loss.
# Adjust num epochs to your dataset. This step is enough to obtain a not bad model.
# 一開始先 freeze YOLO 除了 output layer 以外的 darknet53 backbone 來 train
if True:
    model.compile(optimizer=Adam(lr=1e-3), loss={
        # use custom yolo_loss Lambda layer.
        'yolo_loss': lambda y_true, y_pred: y_pred})

    batch_size = 8
    print('Train on {} samples, val on {} samples, with batch size {}.'.format(num_train, num_val, batch_size))
    # 模型利用 generator 產生的資料做訓練，強烈建議大家去閱讀及理解 data_generator_wrapper 在 train.py 中的實現
    model.fit_generator(data_generator_wrapper(lines[:num_train], batch_size, input_shape, anchors, num_classes),
            steps_per_epoch=max(1, num_train//batch_size),
            validation_data=data_generator_wrapper(lines[num_train:], batch_size, input_shape, anchors, num_classes),
            validation_steps=max(1, num_val//batch_size),
            epochs=50,
            initial_epoch=0,
            callbacks=[logging, checkpoint])
    model.save_weights(log_dir + 'trained_weights_stage_1.h5')















Create YOLOv3 model with 9 anchors and 2 classes.


/usr/local/lib/python3.6/dist-packages/keras/engine/topology.py:3462: UserWarning: Skipping loading of weights for layer conv2d_59 due to mismatch in shape ((1, 1, 1024, 21) vs (255, 1024, 1, 1)).
  weight_values[i].shape))
/usr/local/lib/python3.6/dist-packages/keras/engine/topology.py:3462: UserWarning: Skipping loading of weights for layer conv2d_59 due to mismatch in shape ((21,) vs (255,)).
  weight_values[i].shape))
/usr/local/lib/python3.6/dist-packages/keras/engine/topology.py:3462: UserWarning: Skipping loading of weights for layer conv2d_67 due to mismatch in shape ((1, 1, 512, 21) vs (255, 512, 1, 1)).
  weight_values[i].shape))
/usr/local/lib/python3.6/dist-packages/keras/engine/topology.py:3462: UserWarning: Skipping loading of weights for layer conv2d_67 due to mismatch in shape ((21,) vs (255,)).
  weight_values[i].shape))
/usr/local/lib/python3.6/dist-packages/keras/engine/topology.py:3462: UserWarning: Skipping loading of weights for layer conv2d_75 due to mismatch in 

Load weights model_data/yolo_weights.h5.
Freeze the first 249 layers of total 252 layers.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 327 samples, val on 36 samples, with batch size 8.




Epoch 1/50
40/40 [==============================] - 202s 5s/step - loss: 2973.8929 - val_loss: 368.4119

Epoch 2/50
40/40 [==============================] - 23s 585ms/step - loss: 254.9563 - val_loss: 154.4954
Epoch 3/50
40/40 [==============================] - 26s 649ms/step - loss: 145.7937 - val_loss: 109.3165
Epoch 4/50
40/40 [==============================] - 24s 609ms/step - loss: 105.8616 - val_loss: 81.7074
Epoch 5/50
40/40 [==============================] - 26s 652ms/step - loss: 81.6820 - val_loss: 61.9484
Epoch 6/50
40/40 [==============================] - 26s 655ms/step - loss: 68.2969 - val_loss: 56.2374
Epoch 7/50
40/40 [==============================] - 26s 659ms/step - loss: 58.5743 - val_loss: 47.9052
Epoch 8/50
40/40 [====

In [ ]:
# Unfreeze and continue training, to fine-tune.
# Train longer if the result is not good.
if True:
    # 把所有 layer 都改為 trainable
    for i in range(len(model.layers)):
        model.layers[i].trainable = True
    model.compile(optimizer=Adam(lr=1e-4), loss={'yolo_loss': lambda y_true, y_pred: y_pred}) # recompile to apply the change
    print('Unfreeze all of the layers.')

    batch_size = 4
    print('Train on {} samples, val on {} samples, with batch size {}.'.format(num_train, num_val, batch_size))
    model.fit_generator(data_generator_wrapper(lines[:num_train], batch_size, input_shape, anchors, num_classes),
        steps_per_epoch=max(1, num_train//batch_size),
        validation_data=data_generator_wrapper(lines[num_train:], batch_size, input_shape, anchors, num_classes),
        validation_steps=max(1, num_val//batch_size),
        epochs=100,
        initial_epoch=50,
        callbacks=[logging, checkpoint, reduce_lr, early_stopping])
    model.save_weights(log_dir + 'trained_weights_final.h5')

Unfreeze all of the layers.
Train on 327 samples, val on 36 samples, with batch size 4.
Epoch 51/100
81/81 [==============================] - 47s 578ms/step - loss: 18.2949 - val_loss: 17.0428
Epoch 52/100
81/81 [==============================] - 35s 437ms/step - loss: 16.4169 - val_loss: 15.9702
Epoch 53/100
81/81 [==============================] - 35s 435ms/step - loss: 16.1850 - val_loss: 15.5432
Epoch 54/100
81/81 [==============================] - 35s 437ms/step - loss: 15.4191 - val_loss: 15.8174
Epoch 55/100
81/81 [==============================] - 35s 437ms/step - loss: 15.4222 - val_loss: 15.3478
Epoch 56/100
81/81 [==============================] - 35s 436ms/step - loss: 15.3279 - val_loss: 15.9851
Epoch 57/100
81/81 [==============================] - 35s 437ms/step - loss: 14.8602 - val_loss: 15.3965
Epoch 58/100
81/81 [==============================] - 36s 439ms/step - loss: 15.2344 - val_loss: 16.1566
Epoch 59/100
81/81 [==============================] - 35s 435ms/step - l

In [ ]:
from PIL import Image
image = Image.open('../DataSet/Raccoon/images/raccoon-16.jpg') 
from yolo import YOLO
classes_path = './model_data/finalproject_classes.txt'
anchors_path = './model_data/yolo_anchors.txt'

yolo_model = YOLO(model_path=log_dir + 'trained_weights_final.h5', classes_path=classes_path,anchors_path=anchors_path)
r_image = yolo_model.detect_image(image)
r_image

logs/000/trained_weights_final.h5 model, anchors, and classes loaded.
(416, 416, 3)
Found 1 boxes for img
Raccoon 0.99 (79, 163) (348, 640)
2.6240617479998036


In [13]:
import cv2
import numpy as np
from PIL import Image
import timeit
from yolo import YOLO

log_dir = 'logs/000/' 
classes_path = './model_data/finalproject_classes.txt'
anchors_path = './model_data/yolo_anchors.txt'

yolo_model = YOLO(model_path=log_dir + 'trained_weights_final.h5', classes_path=classes_path,anchors_path=anchors_path)

starttime = timeit.default_timer()
cap = cv2.VideoCapture('../DataSet/Kangaroo.mp4')

# 使用 XVID 編碼
fourcc = cv2.VideoWriter_fourcc(*'XVID')

# 建立 VideoWriter 物件，輸出影片至 output.avi
# FPS 值為 30.0，解析度為 1280*720
out = cv2.VideoWriter('Kangaroo-object-detection.avi', fourcc, 30.0, (1280, 720))

k = 0
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == True:
        # 寫入影格
        image = Image.fromarray(frame)
        image = np.array(yolo_model.detect_image(image))
        out.write(image)
        k += 1
    else:
        break
    
# 釋放所有資源
cap.release()
out.release()

Streaming output truncated to the last 5000 lines.
Found 2 boxes for img
Kangaroo 0.98 (746, 65) (1086, 679)
Raccoon 0.75 (11, 0) (437, 717)
0.05272712899977705
(416, 416, 3)
Found 2 boxes for img
Kangaroo 0.98 (745, 63) (1086, 680)
Raccoon 0.94 (11, 8) (436, 712)
0.04988201200012554
(416, 416, 3)
Found 3 boxes for img
Kangaroo 0.40 (13, 32) (442, 704)
Kangaroo 0.98 (743, 63) (1084, 681)
Raccoon 0.63 (13, 32) (442, 704)
0.0537710989997322
(416, 416, 3)
Found 2 boxes for img
Kangaroo 0.98 (744, 61) (1082, 686)
Raccoon 0.93 (15, 11) (436, 711)
0.05031119699970077
(416, 416, 3)
Found 2 boxes for img
Kangaroo 0.98 (740, 58) (1082, 687)
Raccoon 0.88 (17, 0) (436, 716)
0.05136897800002771
(416, 416, 3)
Found 2 boxes for img
Kangaroo 0.98 (738, 58) (1084, 688)
Raccoon 0.88 (18, 0) (436, 716)
0.05023831800008338
(416, 416, 3)
Found 2 boxes for img
Kangaroo 0.98 (740, 56) (1080, 690)
Raccoon 0.90 (12, 0) (435, 719)
0.050539024000045174
(416, 416, 3)
Found 2 boxes for img
Kangaroo 0.98 (738, 56)